In [7]:
import plotly.express as px
import gtsam
import numpy as np

In [4]:
x = [0.868907, 0.895679, 0.776731, 0.820319, 0.868304]
y = [0.369875, 0.368851, 0.36961, 0.357769, 0.361974]
z = [-0.706912, -0.503801, -0.693484, -0.717067, -0.378006]
x_rb = [0.845988]
y_rb = [0.365616]
z_rb = [-0.599854]
x.extend(x_rb)
y.extend(y_rb)
z.extend(z_rb)

In [5]:
px.scatter_3d(x=x, y=z, z=y, range_x=[0.5, 1], range_y=[-1, 0], range_z=[0, 1])

In [14]:
xyz = []
xyz.append([1.061363,	0.140826,	-0.70525])
xyz.append([1.073549,	0.146109,	-0.500811])
xyz.append([0.968512,	0.144057,	-0.698528])
xyz.append([1.013225,	0.129967,	-0.718491])
xyz.append([1.037059,	0.144586,	-0.37717])
xyz.append([1.030742,	0.141109,	-0.60005])
xyz = np.array(xyz)
px.scatter_3d(x=xyz[:,0], y=xyz[:,2], z=xyz[:,1], range_x=[0.75, 1.25], range_y=[-1, 0], range_z=[0, 1])

In [20]:
marker_3 = np.array([-0.565445,	0.385985,	0.501058])
marker_1 = np.array([-0.536278,	0.388045,	0.589499])
np.linalg.norm(marker_3 - marker_1) / 2

0.04657458526385392

In [23]:
4.34 + 3.1

7.4399999999999995

In [12]:
# ypr (in the camera frame) in degrees - pitch comes out of the top of robot:
# traj1: array([ 0.85217654,  5.37429596, -0.45869616])
# traj2: array([-0.60877149, -2.97709723, -0.25276315])
# traj3: array([-9.04993243, -5.96682431,  0.31206909])
# traj4: array([-1.21250358,  1.24893236, -2.40725622])
# traj5: array([-6.71869945, -2.96622764, -1.11434869])
rot = gtsam.Rot3.Quaternion(0.999667, -0.020888,	0.011118,	-0.010349	)
np.rad2deg(rot.ypr())

array([-1.21250358,  1.24893236, -2.40725622])

In [ ]:
# 1. Calculate vector v1 which goes from marker 1 to marker 2
# 2. Calculate vector v2 which goes from marker 1 to marker 4
# 3. Perform the cross product to obtain v3.
# 4. Use v1, v2, v3 to represent the pose of the rigid body.
# 5. Use linear least squares to estimate the rigid body transform 

In [1]:
import gtsam
import rosbag
import numpy as np
import plotly.express as px

In [2]:
bag = rosbag.Bag('/home/jerredchen/A1_trajs_06-04-2022/A1_bag_2022-03-05-07-01-07.bag')
x = []
y = []
z = []
roll = []
pitch = []
yaw = []
for bag_info in bag.read_messages('/HighState'):
    msg = bag_info[1]
    accel = msg.imu.accelerometer
    gyro = msg.imu.gyroscope
    x.append(accel[0])
    y.append(accel[1])
    z.append(accel[2])
    roll.append(gyro[0])
    pitch.append(gyro[1])
    yaw.append(gyro[2])

In [7]:
px.scatter(y=z)

In [5]:
print(np.mean(x[20000:]))
print(np.mean(y[20000:]))
print(np.mean(z[20000:]))
print(np.mean(roll[20000:]))
print(np.mean(pitch[20000:]))
print(np.mean(yaw[20000:]))

-0.05325466547900562
0.0758941598251736
9.574999714671652
-8.238010928626832e-05
3.3201972823782143e-06
-0.00022398090360004538


In [5]:
print(np.mean(x[18000:19000]))
print(np.mean(y[18000:19000]))
print(np.mean(z[18000:19000]))
print(np.mean(roll[18000:19000]))
print(np.mean(pitch[18000:19000]))
print(np.mean(yaw[18000:19000]))

0.0026961642398964612
0.17724022900266573
9.596930290050805
-0.0021516615400323643
-0.0021505111280130224
0.0015366076509526466


In [15]:
9.81-9.574999714671652

0.23500028532834882

In [16]:
print(np.std(x[20000:])/np.sqrt(500))
print(np.std(y[20000:])/np.sqrt(500))
print(np.std(z[20000:])/np.sqrt(500))
print(np.std(roll[20000:])/np.sqrt(500))
print(np.std(pitch[20000:])/np.sqrt(500))
print(np.std(yaw[20000:])/np.sqrt(500))

0.005171436464324271
0.003346664697299998
0.006605218567845097
0.0004819996688613772
0.0003854767663700326
0.00040906228991292106


In [6]:
print(np.std(x[18000:19000])/np.sqrt(500))
print(np.std(y[18000:19000])/np.sqrt(500))
print(np.std(z[18000:19000])/np.sqrt(500))
print(np.std(roll[18000:19000])/np.sqrt(500))
print(np.std(pitch[18000:19000])/np.sqrt(500))
print(np.std(yaw[18000:19000])/np.sqrt(500))

0.06381007207620515
0.09469571738688635
0.17038279622482055
0.006711957633683302
0.007270743519624283
0.004210133887112282


In [14]:
from gtsam.symbol_shorthand import X, V, B

In [48]:
# Initialize
graph = gtsam.NonlinearFactorGraph()
init_estimates = gtsam.Values()
isam = gtsam.ISAM2(gtsam.ISAM2Params())
results = gtsam.Values()

# Obtain the accelerometer noise standard deviations in all directions, in m/s^2.
# accel_sigmas = [0.005171, 0.00334, 0.00660]
accel_sigmas = [0.05171, 0.0334, 0.0660]

# Obtain the gyroscope noise standard deviations in all directions, in rad/s.
gyro_sigmas = [0.000482, 0.000385, 0.000409],
gyro_sigmas = [0.00482, 0.00385, 0.00409],

# Obtain the integration noise standard deviations in all directions.
integration_sigmas = [1.0e-5, 1.0e-5, 1.0e-5]

# Instantiate covariance matrices according to IMU standard deviations.
accel_noise_cov = np.diag(np.square(accel_sigmas))
gyro_noise_cov = np.diag(np.square(gyro_sigmas))
integration_noise_cov = np.diag(np.square(integration_sigmas))

# Parse the bias parameters and create a GTSAM bias factor.
prior_bias_estimate = [-0.05325, 0.075894, -0.235, -8.238e-5, 3.3202e-6, -0.000224]
accel_bias, gyro_bias = prior_bias_estimate[:3], prior_bias_estimate[3:]
prior_bias_estimate = gtsam.imuBias.ConstantBias(
    accel_bias, gyro_bias)

# Instantiate the PIM parameters to create the PIM object.
pim_parameters = gtsam.PreintegrationParams.MakeSharedU()
pim_parameters.setAccelerometerCovariance(accel_noise_cov)
pim_parameters.setGyroscopeCovariance(gyro_noise_cov)
pim_parameters.setIntegrationCovariance(integration_noise_cov)
pim = gtsam.PreintegratedImuMeasurements(
    pim_parameters, prior_bias_estimate)

# Parse the prior pose estimate and standard deviations parameters.
prior_pose_estimate = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
prior_pose_sigmas = [1.0e-8, 1.0e-8, 1.0e-8, 1.0e-9, 1.0e-9, 1.0e-9]
rpy = np.deg2rad(prior_pose_estimate[3:])
pose_rotation = gtsam.Rot3.Ypr(*(rpy[::-1]))
pose_translation = gtsam.Point3(*prior_pose_estimate[:3])
prior_pose_estimate = gtsam.Pose3(pose_rotation, pose_translation)
prior_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(
    np.concatenate(
        (np.deg2rad(prior_pose_sigmas[3:]), prior_pose_sigmas[:3]))
)

# Create a GTSAM PriorFactorPose3 and add the factor to the factor graph.
prior_pose_factor = gtsam.PriorFactorPose3(
    X(0), prior_pose_estimate, prior_pose_noise)
graph.add(prior_pose_factor)
init_estimates.insert(X(0), prior_pose_factor.prior())

# Parse the velocity parameters and create a GTSAM velocity factor.
prior_vel_estimate = [0.5, 0.0, 0.0]
prior_vel_sigmas = [2.0e-1, 1.0e-3, 1.0e-3]
prior_vel_estimate = np.array(prior_vel_estimate)
prior_vel_noise = gtsam.noiseModel.Diagonal.Sigmas(
    np.array(prior_vel_sigmas))
prior_vel_factor = gtsam.PriorFactorVector(
    V(0), prior_vel_estimate, prior_vel_noise)

# Parse the bias parameters and create a GTSAM bias factor.
prior_bias_estimate = [-0.05325, 0.075894, -0.235, -8.238e-5, 3.3202e-6, -0.000224]
prior_bias_sigmas = [1.0e-3, 1.0e-3, 1.0e-3, 1.0e-5, 1.0e-5, 1.0e-5]
accel_bias, gyro_bias = prior_bias_estimate[:3], prior_bias_estimate[3:]
prior_bias_estimate = gtsam.imuBias.ConstantBias(
    accel_bias, gyro_bias)
prior_bias_noise = gtsam.noiseModel.Diagonal.Sigmas(
    np.array(prior_bias_sigmas))
prior_bias_factor = gtsam.PriorFactorConstantBias(
    B(0), prior_bias_estimate, prior_bias_noise)

# Add the priors to the factor graph.
graph.add(prior_vel_factor)
init_estimates.insert(V(0), prior_vel_factor.prior())
graph.add(prior_bias_factor)
init_estimates.insert(B(0), prior_bias_factor.prior())
isam.update(graph, init_estimates)
results = isam.calculateEstimate()

# Clear graph and initial estimates
graph = gtsam.NonlinearFactorGraph()
init_estimates.clear()

In [49]:
bag = rosbag.Bag('/home/jerredchen/A1_trajs_06-04-2022/A1_bag_2022-03-05-07-01-07.bag')
timestamp = 0
index = 1
for i, bag_info in enumerate(bag.read_messages('/HighState')):
    msg = bag_info[1]
    if i < 3000:
        continue
    if timestamp == 0:
        timestamp = msg.header.stamp.to_sec()
        continue
    if i == 5000:
        break
    current_timestamp = msg.header.stamp.to_sec()
    delta_t = current_timestamp - timestamp
    measured_accel = np.array(msg.imu.accelerometer)
    measured_omega = np.array(msg.imu.gyroscope)
    pim.integrateMeasurement(measured_accel, measured_omega, delta_t)
    timestamp = current_timestamp
    if i % 50 == 0:
        # Calculate the NavState based on the optimized results.
        pose = results.atPose3(X(index - 1))
        vel = results.atVector(V(index - 1))
        navstate = gtsam.NavState(pose, vel)

        # Create an IMU factor and the initial estimates.
        factor = gtsam.ImuFactor(
            X(index - 1),
            V(index - 1),
            X(index),
            V(index),
            B(0),
            pim
        )

        # Initialize the estimates for the pose and velocity by integrating from the previous state.
        predicted_navstate = pim.predict(
            navstate, results.atConstantBias(B(0)))

        graph.add(factor)
        init_estimates.insert(X(index), predicted_navstate.pose())
        init_estimates.insert(V(index), predicted_navstate.velocity())

        
        prior_vel_estimate = [0.5, 0.0, 0.0]
        prior_vel_sigmas = [1.0e-5, 1.0e-5, 1.0e-5]
        prior_vel_estimate = np.array(prior_vel_estimate)
        prior_vel_noise = gtsam.noiseModel.Diagonal.Sigmas(
            np.array(prior_vel_sigmas))
        prior_vel_factor = gtsam.PriorFactorVector(
            V(index), prior_vel_estimate, prior_vel_noise)

        isam.update(graph, init_estimates)
        results = isam.calculateEstimate()

        # Clear graph and initial estimates.
        graph = gtsam.NonlinearFactorGraph()
        init_estimates.clear()
        index += 1

        # Reset the integration of the pim.
        pim.resetIntegration()

In [50]:
results

Values with 81 values:
Value b0: (gtsam::imuBias::ConstantBias)
acc = -0.05325 0.075894   -0.235 gyro = -8.238e-05 3.3202e-06  -0.000224

Value v0: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.5;
	2.49026e-31;
	-5.43423e-33
]

Value v1: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.526813;
	-0.0107126;
	0.0645465
]

Value v2: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.581006;
	-0.0189354;
	0.0692178
]

Value v3: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.641438;
	-0.0351821;
	0.105341
]

Value v4: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.75653;
	0.0681156;
	0.0812442
]

Value v5: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.884885;
	0.0672783;
	0.083658
]

Value v6: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.861094;
	0.0964325;
	0.0899021
]

Value v7: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.888872;
	0.145482;
	0.0824236
]

Value v8: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	0.883053;
	0.204401;
	0.0991279
]

Value v9: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	

In [1]:
import numpy as np

In [20]:
scan = np.array([[0,1,2],[2,1,0]])
cov = np.cov(scan)
eigenval, eigenvec = np.linalg.eig(cov)
normal = eigenvec[:,0] if eigenval[0] < eigenval[1] else eigenvec[:,1]
normal = normal.reshape(-1,1)
P = (normal @ normal.T) / (normal.T @ normal)
P

array([[0.5, 0.5],
       [0.5, 0.5]])

In [12]:
a = np.array([[1,0],[0,0]])
eps = 1e-15 * np.eye(2)
a_prime = a + eps
np.linalg.inv(a_prime) @ a

array([[1., 0.],
       [0., 0.]])